In [ ]:
import pandas as pd
from tensorflow.keras.models import load_model
from joblib import load
# Custom R2 metric function
def r2_keras(output_index):
    def r2(y_true, y_pred):
        SS_res =  K.sum(K.square(y_true[:,output_index] - y_pred[:,output_index]))
        SS_tot = K.sum(K.square(y_true[:,output_index] - K.mean(y_true[:,output_index])))
        r2 = 1 - SS_res/(SS_tot + K.epsilon())
        return r2
    r2._name_ = f"r2_{output_index}"
    return r2
# Load the CSV data
data = pd.read_excel('/content/input (1).xlsx', engine='openpyxl')


# Select the desired columns
selected_columns = [ 'Net Volume', 'Pulp Area', 'Froth surface area', 'Froth thickness', 'Air Flow rate', 'R_inf Ccp', 'R_inf Gn', 'R_inf Po', 'R_inf Sp', 'k_max Ccp', 'k_max Gn', 'k_maxPo', 'k_max Sp', 'Entrainement Savassi parameters', 'Total Solids Flow_Feed', 'Total Liquid Flow_Feed', 'Pulp Volumetric Flow_Feed', 'Solids SG_Feed', 'Pulp SG_Feed', 'Solids Fraction_Feed', 'Cu_Feed', 'Fe_Feed', 'Pb_Feed', 'Zn_Feed' ]  # Add all your columns
input_data = data[selected_columns]

# Load the machine learning model
model = load_model('/content/model7_23.h5', custom_objects={f'r2_{i}': r2_keras(i) for i in range(23)})

# Load the scalers
scalerX = load('/content/scalerX_24.joblib')
scalerY = load('/content/scalerY_23.joblib')

# Scale input features
scaled_input = scalerX.transform(input_data)

# Make predictions
raw_predictions = model.predict(scaled_input)

# Reverse scale the output
final_predictions = scalerY.inverse_transform(raw_predictions)

# Display the predictions
print(final_predictions)


4/4 [==============================] - 0s 5ms/step
[[1.4990569e+02 1.1369913e-01 2.5790203e-01 ... 1.6147146e+01
  2.2406361e+01 9.2632666e+00]
 [1.6806746e+02 9.9072270e-02 2.4768247e-01 ... 1.6405972e+01
  2.1971416e+01 8.8433514e+00]
 [1.4496613e+02 9.3638748e-02 2.4726373e-01 ... 1.6920385e+01
  2.3467773e+01 8.2434530e+00]
 ...
 [1.7792587e+02 1.1489642e-01 2.5919184e-01 ... 1.9633036e+01
  1.7961935e+01 9.3038301e+00]
 [1.4045061e+02 5.6989569e-02 2.4971357e-01 ... 2.0002998e+01
  2.0090961e+01 8.0908251e+00]
 [1.8503870e+02 9.7133338e-02 2.4823910e-01 ... 1.7180803e+01
  2.1128700e+01 7.4423461e+00]]


In [ ]:
import pandas as pd

# Assuming 'final_predictions' contains your predictions
# You can replace it with the actual variable name if it's different
predictions_df = pd.DataFrame(final_predictions, columns=['Air Efficiency', 'Flotation Rate: Cell 1', 'Entrainment: Cell 1', 'Total Solids Flow_Concentrate', 'Total Liquid Flow_Concentrate', 'Pulp Volumetric Flow_Concentrate', 'Solids SG_Concentrate', 'Pulp SG_Concentrate', 'Solids Fraction_Concentrate', 'Total Solids Flow_Tailings', 'Total Liquid Flow_Tailings', 'Pulp Volumetric Flow_Tailings', 'Solids SG_Tailings', 'Pulp SG_Tailings', 'Solids Fraction_Tailings', 'Cu_Tails', 'Fe_Tails', 'Pb_Tails', 'Zn_Tails', 'Cu_Concentrate', 'Fe_Concentrate', 'Pb_Concentrate', 'Zn_Concentrate'])

# Save the DataFrame to a CSV file
predictions_df.head(100).to_excel('/content/output_predictions1.xlsx', index=False)


In [ ]:
import pandas as pd
from sklearn.metrics import r2_score
import numpy as np

# Load the data from the files
ground_truth_df = pd.read_excel('/content/output_data.xlsx')  # Replace with the actual path
predicted_df = pd.read_excel('/content/output_predictions1.xlsx')  # Replace with the actual path

# Ensure both DataFrames have the same number of rows
if ground_truth_df.shape[0] != predicted_df.shape[0]:
    raise ValueError("Number of rows in ground truth and predicted DataFrames must be the same.")

# Extract ground truth and predicted values for all columns
actual_values = ground_truth_df.values
predicted_values = predicted_df.values

# Calculate R-squared for each column
r2_scores = [r2_score(actual_values[:, i], predicted_values[:, i]) for i in range(actual_values.shape[1])]

# Print individual R-squared values and average R-squared
for col, r2 in zip(ground_truth_df.columns, r2_scores):
    print(f'{col} R-squared: {r2:.4f}')

average_r2 = np.mean(r2_scores)
print(f'Average R-squared: {average_r2:.4f}')


Air Efficiency R-squared: 0.0819
 Flotation Rate R-squared: -39.0094
Entrainment: Cell 1 R-squared: -3.4888
Total Solids Flow_Concentrate R-squared: -7.2563
Total Liquid Flow_Concentrate R-squared: -33.7236
Pulp Volumetric Flow_Concentrate R-squared: -32.3083
Solids SG_Concentrate R-squared: -326.5232
Pulp SG_Concentrate R-squared: -3.7331
Solids Fraction_Concentrate R-squared: -4.1581
Cu_Concentrate R-squared: -30412.0073
Pb_Concentrate R-squared: -3285.8236
Fe_Concentrate R-squared: -405.5597
Zn_Concentrate R-squared: -32.7592
Total Solids Flow_Tailings R-squared: -37.7215
Total Liquid Flow_Tailings R-squared: -17.9190
Pulp Volumetric Flow_Tailings R-squared: -51.9005
Solids SG_Tailings R-squared: -39685.3524
Pulp SG_Tailings R-squared: -21369.2481
Solids Fraction_Tailings R-squared: -5.1406
Cu_Tails R-squared: 0.9731
Pb_Tails R-squared: -4.3225
Fe_Tails R-squared: -7.0235
Zn_Tails R-squared: 0.9647
Average R-squared: -4163.6069


In [ ]:
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Load the data from the files
ground_truth_df = pd.read_excel('/content/output_data.xlsx')  # Replace with the actual path
predicted_df = pd.read_excel('/content/output_predictions1.xlsx')

# Ensure both DataFrames have the same number of rows
if ground_truth_df.shape[0] != predicted_df.shape[0]:
    raise ValueError("Number of rows in ground truth and predicted DataFrames must be the same.")

# Extract ground truth and predicted values for all columns
actual_values = ground_truth_df.values
predicted_values = predicted_df.values

# Calculate MAE and RMSE for each column
mae_scores = [mean_absolute_error(actual_values[:, i], predicted_values[:, i]) for i in range(actual_values.shape[1])]
rmse_scores = [mean_squared_error(actual_values[:, i], predicted_values[:, i], squared=False) for i in range(actual_values.shape[1])]

# Prepare results
mae_rmse_results = []
for col, mae, rmse in zip(ground_truth_df.columns, mae_scores, rmse_scores):
    mae_rmse_results.append({
        "Column": col,
        "MAE": mae,
        "RMSE": rmse
    })

mae_rmse_results_df = pd.DataFrame(mae_rmse_results)
average_mae = mae_rmse_results_df["MAE"].mean()
average_rmse = mae_rmse_results_df["RMSE"].mean()
mae_rmse_results_df, average_mae, average_rmse



(                              Column         MAE        RMSE
 0                     Air Efficiency   19.034621   23.076196
 1                     Flotation Rate    0.046829    0.056471
 2                Entrainment: Cell 1    0.004121    0.005952
 3      Total Solids Flow_Concentrate   20.505024   23.785581
 4      Total Liquid Flow_Concentrate  153.417125  162.400365
 5   Pulp Volumetric Flow_Concentrate  149.036335  159.736905
 6              Solids SG_Concentrate    0.682015    0.687394
 7                Pulp SG_Concentrate    1.512823    1.704121
 8        Solids Fraction_Concentrate   55.829303   62.109797
 9                     Cu_Concentrate  111.179829  111.441202
 10                    Pb_Concentrate   23.030543   30.137254
 11                    Fe_Concentrate   34.968395   41.912408
 12                    Zn_Concentrate    6.656152    6.758358
 13        Total Solids Flow_Tailings   92.508156   93.576551
 14        Total Liquid Flow_Tailings   96.931650  100.112849
 15     

In [ ]:
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Load the data from the files
ground_truth_df = pd.read_excel('/content/output_data.xlsx')  # Replace with the actual path
predicted_df = pd.read_excel('/content/output_predictions1.xlsx')

# Ensure both DataFrames have the same number of rows
if ground_truth_df.shape[0] != predicted_df.shape[0]:
    raise ValueError("Number of rows in ground truth and predicted DataFrames must be the same.")

def mean_absolute_percentage_error(y_true, y_pred, threshold=1e-4):  # Augmenter le seuil
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    valid_mask = np.abs(y_true) > threshold
    if np.any(valid_mask):
        return np.mean(np.abs((y_true[valid_mask] - y_pred[valid_mask]) / y_true[valid_mask])) * 100
    else:
        return np.nan


# Calculer le MAPE pour chaque colonne en excluant les valeurs très petites
mape_scores = [mean_absolute_percentage_error(actual_values[:, i], predicted_values[:, i]) for i in range(actual_values.shape[1])]

# Recalculer la moyenne du MAPE en excluant les NaN
average_mape = np.nanmean([mape for mape in mape_scores if not np.isnan(mape)])

# Affichage des résultats
mape_results = [f'{col} MAPE: {mape:.2f}%' for col, mape in zip(ground_truth_df.columns, mape_scores)]
mape_results.append(f'Average MAPE: {average_mape:.2f}%')
for result in mape_results:
    print(result)


Air Efficiency MAPE: 12.04%
 Flotation Rate MAPE: 72.09%
Entrainment: Cell 1 MAPE: 1.68%
Total Solids Flow_Concentrate MAPE: 16.84%
Total Liquid Flow_Concentrate MAPE: 1137.67%
Pulp Volumetric Flow_Concentrate MAPE: 358.00%
Solids SG_Concentrate MAPE: 17.81%
Pulp SG_Concentrate MAPE: 48.56%
Solids Fraction_Concentrate MAPE: 80.50%
Cu_Concentrate MAPE: 2588.06%
Pb_Concentrate MAPE: 589.99%
Fe_Concentrate MAPE: 193.75%
Zn_Concentrate MAPE: 63.01%
Total Solids Flow_Tailings MAPE: 96.71%
Total Liquid Flow_Tailings MAPE: 57.46%
Pulp Volumetric Flow_Tailings MAPE: 97.68%
Solids SG_Tailings MAPE: 315.98%
Pulp SG_Tailings MAPE: 188.88%
Solids Fraction_Tailings MAPE: 10.13%
Cu_Tails MAPE: 1.87%
Pb_Tails MAPE: 19.58%
Fe_Tails MAPE: 28.05%
Zn_Tails MAPE: 1.66%
Average MAPE: 260.78%


In [ ]:
import pandas as pd
from sklearn.metrics import mean_absolute_error

# Load the data from the files
# Load the data from the files
ground_truth_df = pd.read_excel('/content/output_data.xlsx')  # Replace with the actual path
predicted_df = pd.read_excel('/content/output_predictions1.xlsx') # Replace with the actual path

# Ensure both DataFrames have the same number of rows
if ground_truth_df.shape[0] != predicted_df.shape[0]:
    raise ValueError("Number of rows in ground truth and predicted DataFrames must be the same.")

# Columns for which you want to calculate accuracy
selected_columns = ['Air Efficiency', 'Entrainment: Cell 1', 'Total Solids Flow_Concentrate', 'Total Liquid Flow_Concentrate', 'Pulp Volumetric Flow_Concentrate', 'Solids SG_Concentrate', 'Pulp SG_Concentrate', 'Solids Fraction_Concentrate', 'Total Solids Flow_Tailings', 'Total Liquid Flow_Tailings', 'Pulp Volumetric Flow_Tailings', 'Solids SG_Tailings', 'Pulp SG_Tailings', 'Solids Fraction_Tailings', 'Cu_Tails', 'Fe_Tails', 'Pb_Tails', 'Zn_Tails', 'Cu_Concentrate', 'Fe_Concentrate', 'Pb_Concentrate', 'Zn_Concentrate']

# Extract ground truth and predicted values for the specified columns
actual_values = ground_truth_df[selected_columns].values
predicted_values = predicted_df[selected_columns].values

# Calculate Mean Absolute Error (MAE) for each column
mae_values = [mean_absolute_error(actual_values[:, i], predicted_values[:, i]) for i in range(len(selected_columns))]

# Print individual MAE values and average MAE
for col, mae in zip(selected_columns, mae_values):
    print(f'{col} MAE: {mae:.2f}')

average_mae = sum(mae_values) / len(mae_values)
print(f'Average MAE: {average_mae:.2f}')


Air Efficiency MAE: 19.03
Entrainment: Cell 1 MAE: 0.00
Total Solids Flow_Concentrate MAE: 20.51
Total Liquid Flow_Concentrate MAE: 153.42
Pulp Volumetric Flow_Concentrate MAE: 149.04
Solids SG_Concentrate MAE: 0.68
Pulp SG_Concentrate MAE: 1.51
Solids Fraction_Concentrate MAE: 55.83
Total Solids Flow_Tailings MAE: 20.25
Total Liquid Flow_Tailings MAE: 154.11
Pulp Volumetric Flow_Tailings MAE: 143.44
Solids SG_Tailings MAE: 0.71
Pulp SG_Tailings MAE: 1.84
Solids Fraction_Tailings MAE: 65.60
Cu_Tails MAE: 0.76
Fe_Tails MAE: 1.00
Pb_Tails MAE: 18.07
Zn_Tails MAE: 2.18
Cu_Concentrate MAE: 0.78
Fe_Concentrate MAE: 0.67
Pb_Concentrate MAE: 17.97
Zn_Concentrate MAE: 2.46
Average MAE: 37.72


**Accuracy Tails premiére model1**

In [ ]:
import pandas as pd
from sklearn.metrics import mean_absolute_error

# Load the data from the files
# Load the data from the files
ground_truth_df = pd.read_excel('/content/output_data.xlsx')  # Replace with the actual path
predicted_df = pd.read_excel('/content/output_predictions1.xlsx') # Replace with the actual path

# Ensure both DataFrames have the same number of rows
if ground_truth_df.shape[0] != predicted_df.shape[0]:
    raise ValueError("Number of rows in ground truth and predicted DataFrames must be the same.")

# Columns for which you want to calculate accuracy
selected_columns = ['Total Solids Flow_Tailings', 'Total Liquid Flow_Tailings', 'Pulp Volumetric Flow_Tailings', 'Solids SG_Tailings', 'Pulp SG_Tailings', 'Solids Fraction_Tailings', 'Cu_Tails', 'Fe_Tails', 'Pb_Tails', 'Zn_Tails']

# Extract ground truth and predicted values for the specified columns
actual_values = ground_truth_df[selected_columns].values
predicted_values = predicted_df[selected_columns].values

# Calculate Mean Absolute Error (MAE) for each column
mae_values = [mean_absolute_error(actual_values[:, i], predicted_values[:, i]) for i in range(len(selected_columns))]

# Print individual MAE values and average MAE
for col, mae in zip(selected_columns, mae_values):
    print(f'{col} MAE: {mae:.2f}')

average_mae = sum(mae_values) / len(mae_values)
print(f'Average MAE: {average_mae:.2f}')


Total Solids Flow_Tailings MAE: 20.25
Total Liquid Flow_Tailings MAE: 154.11
Pulp Volumetric Flow_Tailings MAE: 143.44
Solids SG_Tailings MAE: 0.71
Pulp SG_Tailings MAE: 1.84
Solids Fraction_Tailings MAE: 65.60
Cu_Tails MAE: 0.76
Fe_Tails MAE: 1.00
Pb_Tails MAE: 18.07
Zn_Tails MAE: 2.18
Average MAE: 40.80


In [ ]:
import pandas as pd
from sklearn.metrics import mean_absolute_error

# Load the data from the files
# Load the data from the files
ground_truth_df = pd.read_excel('/content/output_data.xlsx')  # Replace with the actual path
predicted_df = pd.read_excel('/content/output_predictions1.xlsx') # Replace with the actual path

# Ensure both DataFrames have the same number of rows
if ground_truth_df.shape[0] != predicted_df.shape[0]:
    raise ValueError("Number of rows in ground truth and predicted DataFrames must be the same.")

# Columns for which you want to calculate accuracy
selected_columns = ['Total Solids Flow_Concentrate', 'Total Liquid Flow_Concentrate', 'Pulp Volumetric Flow_Concentrate', 'Solids SG_Concentrate', 'Pulp SG_Concentrate', 'Solids Fraction_Concentrate', 'Cu_Tails',  'Cu_Concentrate', 'Fe_Concentrate', 'Pb_Concentrate', 'Zn_Concentrate']

# Extract ground truth and predicted values for the specified columns
actual_values = ground_truth_df[selected_columns].values
predicted_values = predicted_df[selected_columns].values

# Calculate Mean Absolute Error (MAE) for each column
mae_values = [mean_absolute_error(actual_values[:, i], predicted_values[:, i]) for i in range(len(selected_columns))]

# Print individual MAE values and average MAE
for col, mae in zip(selected_columns, mae_values):
    print(f'{col} MAE: {mae:.2f}')

average_mae = sum(mae_values) / len(mae_values)
print(f'Average MAE: {average_mae:.2f}')


Total Solids Flow_Concentrate MAE: 20.51
Total Liquid Flow_Concentrate MAE: 153.42
Pulp Volumetric Flow_Concentrate MAE: 149.04
Solids SG_Concentrate MAE: 0.68
Pulp SG_Concentrate MAE: 1.51
Solids Fraction_Concentrate MAE: 55.83
Cu_Tails MAE: 0.76
Cu_Concentrate MAE: 0.78
Fe_Concentrate MAE: 0.67
Pb_Concentrate MAE: 17.97
Zn_Concentrate MAE: 2.46
Average MAE: 36.69


**model 3**

In [ ]:
import pandas as pd
from tensorflow.keras.models import load_model
from joblib import load
# Custom R2 metric function
def r2_keras(output_index):
    def r2(y_true, y_pred):
        SS_res =  K.sum(K.square(y_true[:,output_index] - y_pred[:,output_index]))
        SS_tot = K.sum(K.square(y_true[:,output_index] - K.mean(y_true[:,output_index])))
        r2 = 1 - SS_res/(SS_tot + K.epsilon())
        return r2
    r2._name_ = f"r2_{output_index}"
    return r2
# Load the CSV data
data = pd.read_excel('/content/input3.xlsx', engine='openpyxl')


# Select the desired columns
selected_columns = ['Net Volume','Pulp Area', 'Froth surface area', 'Froth thickness', 'Air Flow rate', 'R_inf Ccp', 'R_inf Gn', 'R_inf Po', 'R_inf Sp', 'k_max Ccp', 'k_max Gn', 'k_maxPo', 'k_max Sp', 'Entrainement Savassi parameters', 'Total Solids Flow_Feed','Total Liquid Flow_Feed','Pulp Volumetric Flow_Feed', 'Solids SG_Feed', 'Pulp SG_Feed', 'Solids Fraction_Feed', 'Cu_Feed', 'Fe_Feed','Pb_Feed', 'Zn_Feed' ]  # Add all your columns
input_data = data[selected_columns]

# Load the machine learning model
model_3 = load_model('/content/model7_23.h5', custom_objects={f'r2_{i}': r2_keras(i) for i in range(23)})

# Load the scalers
scalerX = load('/content/scalerX_24.joblib')
scalerY = load('/content/scalerY_23.joblib')

# Scale input features
scaled_input = scalerX.transform(input_data)

# Make predictions
raw_predictions = model_3.predict(scaled_input)

# Reverse scale the output
final_predictions3 = scalerY.inverse_transform(raw_predictions)

# Display the predictions
print(final_predictions3)


4/4 [==============================] - 0s 3ms/step
[[4.1501755e+02 3.2046092e-01 5.1672286e-01 ... 2.5405834e+01
  1.2284738e+01 1.5849674e+01]
 [7.1928119e+02 6.6778570e-01 7.8044552e-01 ... 3.4462818e+01
  8.7103243e+00 2.2986769e+01]
 [4.1801306e+02 3.4595144e-01 5.3281832e-01 ... 2.6603273e+01
  1.1993875e+01 1.5754798e+01]
 ...
 [5.4662585e+02 4.7401398e-01 6.3193321e-01 ... 3.0585390e+01
  1.0446027e+01 1.8562387e+01]
 [2.3832800e+02 1.5576310e-01 3.5401505e-01 ... 2.2218422e+01
  1.2965219e+01 1.1262331e+01]
 [6.6577222e+02 6.3063371e-01 7.3443002e-01 ... 3.3593712e+01
  9.1655388e+00 2.1077110e+01]]


In [ ]:
import pandas as pd

# Assuming 'final_predictions' contains your predictions
# You can replace it with the actual variable name if it's different
predictions_df = pd.DataFrame(final_predictions, columns=['Air Efficiency', 'Flotation Rate: Cell 3', 'Entrainment: Cell 3', 'Total Solids Flow_Concentrate', 'Total Liquid Flow_Concentrate', 'Pulp Volumetric Flow_Concentrate', 'Solids SG_Concentrate', 'Pulp SG_Concentrate', 'Solids Fraction_Concentrate', 'Total Solids Flow_Tailings', 'Total Liquid Flow_Tailings', 'Pulp Volumetric Flow_Tailings', 'Solids SG_Tailings', 'Pulp SG_Tailings', 'Solids Fraction_Tailings', 'Cu_Tails', 'Fe_Tails', 'Pb_Tails', 'Zn_Tails', 'Cu_Concentrate', 'Fe_Concentrate', 'Pb_Concentrate', 'Zn_Concentrate'])

# Save the DataFrame to a CSV file
predictions_df.head(100).to_excel('/content/output_predictions3.xlsx', index=False)


**accuracy tailde model 3**

In [ ]:
import pandas as pd
from sklearn.metrics import mean_absolute_error

# Load the data from the files
# Load the data from the files
ground_truth_df = pd.read_excel('/content/output_data_3.xlsx')  # Replace with the actual path
predicted_df = pd.read_excel('/content/output_predictions3.xlsx') # Replace with the actual path

# Ensure both DataFrames have the same number of rows
if ground_truth_df.shape[0] != predicted_df.shape[0]:
    raise ValueError("Number of rows in ground truth and predicted DataFrames must be the same.")

# Columns for which you want to calculate accuracy
selected_columns = ['Total Solids Flow_Tailings', 'Total Liquid Flow_Tailings', 'Pulp Volumetric Flow_Tailings', 'Solids SG_Tailings', 'Pulp SG_Tailings', 'Solids Fraction_Tailings', 'Cu_Tails', 'Fe_Tails', 'Pb_Tails', 'Zn_Tails']

# Extract ground truth and predicted values for the specified columns
actual_values = ground_truth_df[selected_columns].values
predicted_values = predicted_df[selected_columns].values

# Calculate Mean Absolute Error (MAE) for each column
mae_values = [mean_absolute_error(actual_values[:, i], predicted_values[:, i]) for i in range(len(selected_columns))]

# Print individual MAE values and average MAE
for col, mae in zip(selected_columns, mae_values):
    print(f'{col} MAE: {mae:.2f}')

average_mae = sum(mae_values) / len(mae_values)
print(f'Average MAE: {average_mae:.2f}')


Total Solids Flow_Tailings MAE: 91.06
Total Liquid Flow_Tailings MAE: 33.14
Pulp Volumetric Flow_Tailings MAE: 26.02
Solids SG_Tailings MAE: 0.03
Pulp SG_Tailings MAE: 1.82
Solids Fraction_Tailings MAE: 62.51
Cu_Tails MAE: 0.09
Fe_Tails MAE: 0.52
Pb_Tails MAE: 1.12
Zn_Tails MAE: 0.18
Average MAE: 21.65


In [ ]:
import pandas as pd
from sklearn.metrics import mean_absolute_error

# Load the data from the files
# Load the data from the files
ground_truth_df = pd.read_excel('/content/output_data_3.xlsx')  # Replace with the actual path
predicted_df = pd.read_excel('/content/output_predictions3.xlsx') # Replace with the actual path

# Ensure both DataFrames have the same number of rows
if ground_truth_df.shape[0] != predicted_df.shape[0]:
    raise ValueError("Number of rows in ground truth and predicted DataFrames must be the same.")

# Columns for which you want to calculate accuracy
selected_columns = ['Air Efficiency', 'Flotation Rate: Cell 3', 'Entrainment: Cell 3', 'Total Solids Flow_Concentrate', 'Total Liquid Flow_Concentrate', 'Pulp Volumetric Flow_Concentrate', 'Solids SG_Concentrate', 'Pulp SG_Concentrate', 'Solids Fraction_Concentrate', 'Total Solids Flow_Tailings', 'Total Liquid Flow_Tailings', 'Pulp Volumetric Flow_Tailings', 'Solids SG_Tailings', 'Pulp SG_Tailings', 'Solids Fraction_Tailings', 'Cu_Tails', 'Fe_Tails', 'Pb_Tails', 'Zn_Tails', 'Cu_Concentrate', 'Fe_Concentrate', 'Pb_Concentrate', 'Zn_Concentrate']

# Extract ground truth and predicted values for the specified columns
actual_values = ground_truth_df[selected_columns].values
predicted_values = predicted_df[selected_columns].values

# Calculate Mean Absolute Error (MAE) for each column
mae_values = [mean_absolute_error(actual_values[:, i], predicted_values[:, i]) for i in range(len(selected_columns))]

# Print individual MAE values and average MAE
for col, mae in zip(selected_columns, mae_values):
    print(f'{col} MAE: {mae:.2f}')

average_mae = sum(mae_values) / len(mae_values)
print(f'Average MAE: {average_mae:.2f}')


Air Efficiency MAE: 57.85
Flotation Rate: Cell 3 MAE: 0.07
Entrainment: Cell 3 MAE: 0.02
Total Solids Flow_Concentrate MAE: 31.19
Total Liquid Flow_Concentrate MAE: 47.61
Pulp Volumetric Flow_Concentrate MAE: 59.78
Solids SG_Concentrate MAE: 0.31
Pulp SG_Concentrate MAE: 0.01
Solids Fraction_Concentrate MAE: 0.64
Total Solids Flow_Tailings MAE: 91.06
Total Liquid Flow_Tailings MAE: 33.14
Pulp Volumetric Flow_Tailings MAE: 26.02
Solids SG_Tailings MAE: 0.03
Pulp SG_Tailings MAE: 1.82
Solids Fraction_Tailings MAE: 62.51
Cu_Tails MAE: 0.09
Fe_Tails MAE: 0.52
Pb_Tails MAE: 1.12
Zn_Tails MAE: 0.18
Cu_Concentrate MAE: 0.28
Fe_Concentrate MAE: 0.30
Pb_Concentrate MAE: 2.55
Zn_Concentrate MAE: 0.78
Average MAE: 18.17


**model 5**

In [ ]:
import pandas as pd
from tensorflow.keras.models import load_model
from joblib import load
# Custom R2 metric function
def r2_keras(output_index):
    def r2(y_true, y_pred):
        SS_res =  K.sum(K.square(y_true[:,output_index] - y_pred[:,output_index]))
        SS_tot = K.sum(K.square(y_true[:,output_index] - K.mean(y_true[:,output_index])))
        r2 = 1 - SS_res/(SS_tot + K.epsilon())
        return r2
    r2._name_ = f"r2_{output_index}"
    return r2
# Load the CSV data
data = pd.read_excel('/content/input5.xlsx', engine='openpyxl')


# Select the desired columns
selected_columns = ['Net Volume','Pulp Area', 'Froth surface area', 'Froth thickness', 'Air Flow rate', 'R_inf Ccp', 'R_inf Gn', 'R_inf Po', 'R_inf Sp', 'k_max Ccp', 'k_max Gn', 'k_maxPo', 'k_max Sp', 'Entrainement Savassi parameters', 'Total Solids Flow_Feed','Total Liquid Flow_Feed','Pulp Volumetric Flow_Feed', 'Solids SG_Feed', 'Pulp SG_Feed', 'Solids Fraction_Feed', 'Cu_Feed', 'Fe_Feed','Pb_Feed', 'Zn_Feed' ]  # Add all your columns
input_data = data[selected_columns]

# Load the machine learning model
model_3 = load_model('/content/model7_23.h5', custom_objects={f'r2_{i}': r2_keras(i) for i in range(23)})

# Load the scalers
scalerX = load('/content/scalerX_24.joblib')
scalerY = load('/content/scalerY_23.joblib')

# Scale input features
scaled_input = scalerX.transform(input_data)

# Make predictions
raw_predictions = model_3.predict(scaled_input)

# Reverse scale the output
final_predictions3 = scalerY.inverse_transform(raw_predictions)

# Display the predictions
print(final_predictions3)


4/4 [==============================] - 0s 5ms/step
[[4.1501755e+02 3.2046092e-01 5.1672286e-01 ... 2.5405834e+01
  1.2284738e+01 1.5849674e+01]
 [7.1928119e+02 6.6778570e-01 7.8044552e-01 ... 3.4462818e+01
  8.7103243e+00 2.2986769e+01]
 [4.1801306e+02 3.4595144e-01 5.3281832e-01 ... 2.6603273e+01
  1.1993875e+01 1.5754798e+01]
 ...
 [5.4662585e+02 4.7401398e-01 6.3193321e-01 ... 3.0585390e+01
  1.0446027e+01 1.8562387e+01]
 [2.3832800e+02 1.5576310e-01 3.5401505e-01 ... 2.2218422e+01
  1.2965219e+01 1.1262331e+01]
 [6.6577222e+02 6.3063371e-01 7.3443002e-01 ... 3.3593712e+01
  9.1655388e+00 2.1077110e+01]]


In [ ]:
import pandas as pd

# Assuming 'final_predictions' contains your predictions
# You can replace it with the actual variable name if it's different
predictions_df = pd.DataFrame(final_predictions, columns=['Air Efficiency', 'Flotation Rate: Cell 5', 'Entrainment: Cell 5', 'Total Solids Flow_Concentrate', 'Total Liquid Flow_Concentrate', 'Pulp Volumetric Flow_Concentrate', 'Solids SG_Concentrate', 'Pulp SG_Concentrate', 'Solids Fraction_Concentrate', 'Total Solids Flow_Tailings', 'Total Liquid Flow_Tailings', 'Pulp Volumetric Flow_Tailings', 'Solids SG_Tailings', 'Pulp SG_Tailings', 'Solids Fraction_Tailings', 'Cu_Tails', 'Fe_Tails', 'Pb_Tails', 'Zn_Tails', 'Cu_Concentrate', 'Fe_Concentrate', 'Pb_Concentrate', 'Zn_Concentrate'])

# Save the DataFrame to a CSV file
predictions_df.head(100).to_excel('/content/output_predictions5.xlsx', index=False)


***calcule accuracy model 5***

In [ ]:
import pandas as pd
from sklearn.metrics import mean_absolute_error

# Load the data from the files
# Load the data from the files
ground_truth_df = pd.read_excel('/content/output_data_5.xlsx')  # Replace with the actual path
predicted_df = pd.read_excel('/content/output_predictions5.xlsx') # Replace with the actual path

# Ensure both DataFrames have the same number of rows
if ground_truth_df.shape[0] != predicted_df.shape[0]:
    raise ValueError("Number of rows in ground truth and predicted DataFrames must be the same.")

# Columns for which you want to calculate accuracy
selected_columns = ['Total Solids Flow_Tailings', 'Total Liquid Flow_Tailings', 'Pulp Volumetric Flow_Tailings', 'Solids SG_Tailings', 'Pulp SG_Tailings', 'Solids Fraction_Tailings', 'Cu_Tails', 'Fe_Tails', 'Pb_Tails', 'Zn_Tails']

# Extract ground truth and predicted values for the specified columns
actual_values = ground_truth_df[selected_columns].values
predicted_values = predicted_df[selected_columns].values

# Calculate Mean Absolute Error (MAE) for each column
mae_values = [mean_absolute_error(actual_values[:, i], predicted_values[:, i]) for i in range(len(selected_columns))]

# Print individual MAE values and average MAE
for col, mae in zip(selected_columns, mae_values):
    print(f'{col} MAE: {mae:.2f}')

average_mae = sum(mae_values) / len(mae_values)
print(f'Average MAE: {average_mae:.2f}')


Total Solids Flow_Tailings MAE: 92.23
Total Liquid Flow_Tailings MAE: 31.55
Pulp Volumetric Flow_Tailings MAE: 25.16
Solids SG_Tailings MAE: 0.02
Pulp SG_Tailings MAE: 1.81
Solids Fraction_Tailings MAE: 61.86
Cu_Tails MAE: 0.11
Fe_Tails MAE: 0.23
Pb_Tails MAE: 0.87
Zn_Tails MAE: 0.39
Average MAE: 21.42


In [ ]:
import pandas as pd
from sklearn.metrics import mean_absolute_error

# Load the data from the files
# Load the data from the files
ground_truth_df = pd.read_excel('/content/output_data_5.xlsx')  # Replace with the actual path
predicted_df = pd.read_excel('/content/output_predictions5.xlsx') # Replace with the actual path

# Ensure both DataFrames have the same number of rows
if ground_truth_df.shape[0] != predicted_df.shape[0]:
    raise ValueError("Number of rows in ground truth and predicted DataFrames must be the same.")

# Columns for which you want to calculate accuracy
selected_columns = ['Air Efficiency', 'Flotation Rate: Cell 5', 'Entrainment: Cell 5', 'Total Solids Flow_Concentrate', 'Total Liquid Flow_Concentrate', 'Pulp Volumetric Flow_Concentrate', 'Solids SG_Concentrate', 'Pulp SG_Concentrate', 'Solids Fraction_Concentrate', 'Total Solids Flow_Tailings', 'Total Liquid Flow_Tailings', 'Pulp Volumetric Flow_Tailings', 'Solids SG_Tailings', 'Pulp SG_Tailings', 'Solids Fraction_Tailings', 'Cu_Tails', 'Fe_Tails', 'Pb_Tails', 'Zn_Tails', 'Cu_Concentrate', 'Fe_Concentrate', 'Pb_Concentrate', 'Zn_Concentrate']

# Extract ground truth and predicted values for the specified columns
actual_values = ground_truth_df[selected_columns].values
predicted_values = predicted_df[selected_columns].values

# Calculate Mean Absolute Error (MAE) for each column
mae_values = [mean_absolute_error(actual_values[:, i], predicted_values[:, i]) for i in range(len(selected_columns))]

# Print individual MAE values and average MAE
for col, mae in zip(selected_columns, mae_values):
    print(f'{col} MAE: {mae:.2f}')

average_mae = sum(mae_values) / len(mae_values)
print(f'Average MAE: {average_mae:.2f}')


Air Efficiency MAE: 170.00
Flotation Rate: Cell 5 MAE: 0.11
Entrainment: Cell 5 MAE: 0.18
Total Solids Flow_Concentrate MAE: 101.22
Total Liquid Flow_Concentrate MAE: 177.66
Pulp Volumetric Flow_Concentrate MAE: 204.78
Solids SG_Concentrate MAE: 0.83
Pulp SG_Concentrate MAE: 0.02
Solids Fraction_Concentrate MAE: 1.51
Total Solids Flow_Tailings MAE: 92.23
Total Liquid Flow_Tailings MAE: 31.55
Pulp Volumetric Flow_Tailings MAE: 25.16
Solids SG_Tailings MAE: 0.02
Pulp SG_Tailings MAE: 1.81
Solids Fraction_Tailings MAE: 61.86
Cu_Tails MAE: 0.11
Fe_Tails MAE: 0.23
Pb_Tails MAE: 0.87
Zn_Tails MAE: 0.39
Cu_Concentrate MAE: 2.21
Fe_Concentrate MAE: 7.83
Pb_Concentrate MAE: 25.83
Zn_Concentrate MAE: 3.02
Average MAE: 39.54
